In the previous session we trained a model for predicting churn and evaluated it. Our model is still in Jupyter notebook. Now let's deploy it - save it and use it.

“Deploy the model” means taking a trained machine-learning model and making it available for real-world use, so that new data can be sent to it and it can return predictions.

This usually involves putting the model into a production environment—such as a web service, API, cloud platform, or application—so that users or other systems can call it to get predictions.

**Code from the previous steps**

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
# Data preparation
df = pd.read_csv('https://raw.githubusercontent.com/ekhlebus/ml-zoomcamp/refs/heads/main/03-classification_churn_prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv')

#df = pd.read_csv('data-week-3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [3]:
# Data splitting
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [4]:
# Variables we have
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [5]:
# Model training function
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=5000)
    model.fit(X_train, y_train)
    
    return dv, model

In [6]:
# Prediction function
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [7]:
# Parameters of the model
C = 1.0
n_splits = 5 # number of folds for cross-validation

In [8]:
# Cross-validation for training model and evaluating
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s, %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0, 0.842 +- 0.007


In [9]:
scores

[0.8443375120691413,
 0.8449977651966627,
 0.8333012241621514,
 0.8347768913474451,
 0.8518166985660947]

In [10]:
# Train final model on the full training data
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values

# Look at the AUC score
auc = roc_auc_score(y_test, y_pred)
auc

0.8584032088573997

## Save the model

Save the model in order to be able to use it later

In [11]:
# pickle is a built-in python library for saving and loading python objects
import pickle

Now we need to take our model and write it to a file.

In [12]:
# Create the output file name
output_file = f'churn_prediction_model_C={C}.bin'
output_file

'churn_prediction_model_C=1.0.bin'

In [13]:
f_out = open(output_file, 'wb')  # 'wb' means we want to write to this file in binary mode
pickle.dump((dv, model), f_out) # We need to save both the DictVectorizer and the model to be able to translate the user/customer into a feature matrix 
f_out.close() # Close the file is very important

In [14]:
!ls -lh *.bin

-rw-rw-rw- 1 codespace codespace 2.5K Dec 10 22:19 'churn_prediction_model_C=1.0.bin'
-rw-rw-rw- 1 codespace codespace 2.5K Dec 10 22:17 'model_C=1.0.bin'


In [15]:
# Another way to save a model using "with" statement
# Here we don't need to explicitly close the file (it will be closed automatically when we exit the "with" block)
with open(output_file, 'wb') as f_out: 
    pickle.dump((dv, model), f_out)

## Load the model

(!) Before that restart the kernel and pretend that you are in a different process!

In [1]:
import pickle

In [2]:
input_file = 'churn_prediction_model_C=1.0.bin'

In [3]:
with open(input_file, 'rb') as f_in: 
    dv, model = pickle.load(f_in)

In [4]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=5000))

In [5]:
# This is an example customer data for prediction
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [6]:
# At first we need to convert the customer into a feature matrix
X = dv.transform([customer])

In [ ]:
# Getting the probability of churn for this particular customer
y_pred = model.predict_proba(X)[0, 1]
y_pred

np.float64(0.6275953527536646)

In [8]:
print('input:', customer)
print('output:', y_pred)

input: {'gender': 'female', 'seniorcitizen': 0, 'partner': 'yes', 'dependents': 'no', 'phoneservice': 'no', 'multiplelines': 'no_phone_service', 'internetservice': 'dsl', 'onlinesecurity': 'no', 'onlinebackup': 'yes', 'deviceprotection': 'no', 'techsupport': 'no', 'streamingtv': 'no', 'streamingmovies': 'no', 'contract': 'month-to-month', 'paperlessbilling': 'yes', 'paymentmethod': 'electronic_check', 'tenure': 1, 'monthlycharges': 29.85, 'totalcharges': 29.85}
output: 0.6275953527536646


## Making requests

If we don't want to run Jupyter notebook every time when we want to use the model we can make a python script for that.

In [40]:
import requests

In [41]:
url = 'http://localhost:9696/predict'

In [42]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'two_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [ ]:
response = requests.post(url, json=customer).json()

In [39]:
response

{'churn': True, 'churn_probability': 0.5133820686195286}

In [27]:
if response['churn']:
    print('sending email to', 'asdx-123d')

sending email to asdx-123d
